# TF-TRT Inference From Saved Model with Tensorflow <= 1.13

In this notebook, we demonstrate the process to create a TF-TRT optimized model from a Tensorflow *saved model*. This notebook has been successfully tested with the NVIDIA NGC Tensorflow 1.13 container `nvcr.io/nvidia/tensorflow:19.04-py3` that can be downloaded freely from http://ngc.nvidia.com. We note that from TF 1.14 onwards, a cleaner API interface for INT8 conversion has been implemented.


## Saved model
We will run this demonstration with a saved Resnet-v1-50 model stored at `/path/to/saved_model`.

## Data
The INT8 calibration process requires access to a small but representative sample of real training or valiation data.

We will use the ImageNet dataset that is stored in TFrecords format. Google provide an excellent all-in-one script for downloading and preparing the ImageNet dataset at 

https://github.com/tensorflow/models/blob/master/research/inception/inception/data/download_and_preprocess_imagenet.sh.

 
## Quick start

To run this notebook, start the NGC TF container, providing correct path to the ImageNet validation data `/path/to/image_net` and the folder `/path/to/saved_model` containing the TF saved model:

```bash
nvidia-docker run --rm -it -p 8888:8888 -v /path/to/image_net:/data  -v /path/to/saved_model:/saved_model --name TFTRT nvcr.io/nvidia/tensorflow:19.04-py3
```

This repository can be cloned to `/workspace` within the container with 
```bash
git clone https://github.com/vinhngx/tftrt-examples
```

Within the container, we then start Jupyter notebook with:

```bash
jupyter notebook --ip 0.0.0.0 --port 8888  --allow-root
```

Connect to Jupyter notebook web interface on your host http://localhost:8888.


We first install some extra packages and external dependencies needed for, e.g. preprocessing ImageNet data. 

In [ ]:
%%bash
pushd /workspace/nvidia-examples/tensorrt/tftrt/examples/object_detection/ 
bash install_dependencies.sh;
popd

In [2]:
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Data
We verify that the correct data folder has been mounted and validation data files of the form `validation-00xxx-of-00128` are available.

In [3]:
VALIDATION_DATA_DIR = "/data"

def get_files(data_dir, filename_pattern):
    if data_dir == None:
        return []
    files = tf.gfile.Glob(os.path.join(data_dir, filename_pattern))
    if files == []:
        raise ValueError('Can not find any files in {} with '
                         'pattern "{}"'.format(data_dir, filename_pattern))
    return files

calibration_files = get_files(VALIDATION_DATA_DIR, 'validation*')
print('There are %d calibration files. \n%s\n%s\n...'%(len(calibration_files), calibration_files[0], calibration_files[-1]))

There are 128 calibration files. 
/data/validation-00114-of-00128
/data/validation-00094-of-00128
...


## Helper functions
We define a few helper functions to read and preprocess Imagenet data from TFRecord files. 

In [5]:
def deserialize_image_record(record):
    feature_map = {
        'image/encoded':          tf.FixedLenFeature([ ], tf.string, ''),
        'image/class/label':      tf.FixedLenFeature([1], tf.int64,  -1),
        'image/class/text':       tf.FixedLenFeature([ ], tf.string, ''),
        'image/object/bbox/xmin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/xmax': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymax': tf.VarLenFeature(dtype=tf.float32)
    }
    with tf.name_scope('deserialize_image_record'):
        obj = tf.parse_single_example(record, feature_map)
        imgdata = obj['image/encoded']
        label   = tf.cast(obj['image/class/label'], tf.int32)
        bbox    = tf.stack([obj['image/object/bbox/%s'%x].values
                            for x in ['ymin', 'xmin', 'ymax', 'xmax']])
        bbox = tf.transpose(tf.expand_dims(bbox, 0), [0,2,1])
        text    = obj['image/class/text']
        return imgdata, label, bbox, text

In [6]:
from preprocessing import vgg_preprocessing
def preprocess(record):
        # Parse TFRecord
        imgdata, label, bbox, text = deserialize_image_record(record)
        label -= 1 # Change to 0-based (don't use background class)
        try:    image = tf.image.decode_jpeg(imgdata, channels=3, fancy_upscaling=False, dct_method='INTEGER_FAST')
        except: image = tf.image.decode_png(imgdata, channels=3)

        image = vgg_preprocessing.preprocess_image(image, 224, 224, is_training=False)
        return image, label

## Benchmarking naitive Tensorflow model

In [8]:
SAVED_MODEL_DIR =  "/saved_model/resnet_v1_50_savedmodel/1"

We employ `saved_model_cli` to inspect the inputs and outputs of the model.

In [9]:
!saved_model_cli show --all --dir $SAVED_MODEL_DIR


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 224, 224, 3)
        name: import/input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1000)
        name: import/resnet_v1_50/predictions/Softmax:0
  Method name is: tensorflow/serving/predict


In [11]:
INPUT_TENSOR = 'import/input:0'
OUTPUT_TENSOR = 'import/resnet_v1_50/predictions/Softmax:0'
    

In [12]:
#Define some global variables
BATCH_SIZE = 128

In [22]:
with tf.Session(graph=tf.Graph()) as sess:
    print('Preparing dataset...')
    dataset = tf.data.TFRecordDataset(calibration_files)    
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    
    print('Loading naitive model...')
    tf.saved_model.loader.load(
        sess, [tf.saved_model.tag_constants.SERVING], SAVED_MODEL_DIR)

    print('Warming up for 10 batches...')
    for _ in range (10):
        image_data = sess.run(next_element)    
        img = image_data[0]
        output = sess.run([OUTPUT_TENSOR], feed_dict={INPUT_TENSOR: img})
    
    num_hits = 0
    num_predict = 0
    start_time = time.time()
    try:
        while True:        
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run(output_node[0].outputs[0], feed_dict={INPUT_TENSOR: img})
            prediction = np.argmax(output, axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    except tf.errors.OutOfRangeError as e:
        pass
            
    print('Naitive Tensorflow Accuracy: %.2f%%'%(100*num_hits/num_predict)) 
    print('Naitive Tensorflow Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Warming up for 10 batches...
Naitive Tensorflow Accuracy: 75.25%
Naitive Tensorflow Inference speed: 1220.02 samples/s


## Benchmarking TF-TRT FP16 inference engine

In [15]:
#First, we extract the FP32 graphdef
tf.reset_default_graph()
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(
        sess, [tf.saved_model.tag_constants.SERVING], SAVED_MODEL_DIR)
    fp32_graph_def = sess.graph_def
len(fp32_graph_def.SerializeToString()) 

#Now we create the TFTRT FP16 engine
trt_fp16_Graph = trt.create_inference_graph(
        input_graph_def=fp32_graph_def,
        outputs=[OUTPUT_TENSOR],
        max_batch_size=BATCH_SIZE,
        max_workspace_size_bytes=1<<32,
        precision_mode='FP16')
    
with tf.Session(graph=tf.Graph()) as sess:
    print('Preparing dataset...')
    dataset = tf.data.TFRecordDataset(calibration_files)    
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    print('Loading TF-TRT FP16 inference engine...)
    output_node = tf.import_graph_def(
        trt_fp16_Graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

    print('Warming up for 10 batches...')
    for _ in range (10):
        image_data = sess.run(next_element)    
        img = image_data[0]
        output = sess.run([OUTPUT_TENSOR], feed_dict={INPUT_TENSOR: img})

    print('Benchmarking FP16...')
    num_hits = 0
    num_predict = 0
    start_time = time.time()
    try:
        while True:        
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run(output_node, feed_dict={INPUT_TENSOR: img})
            prediction = np.argmax(output[0], axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    except tf.errors.OutOfRangeError as e:
        pass
            
    print('FP16 TF-TRT Accuracy: %.2f%%'%(100*num_hits/num_predict))
    print('FP16 TF-TRT Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
INFO:tensorflow:Running against TensorRT version 5.1.2
Warming up for 10 batches...
Benchmarking FP16...
FP16 TF-TRT Accuracy: 75.28%
FP16 TF-TRT Inference speed: 1327.73 samples/s


## Creating TFTRT INT8 inference model

Creating TF-TRT INT8 inference model requires two steps:

- Step 1: creating the calibration graph, and run some training data through that graph for INT-8 calibration.

- Step 2: converting the calibration graph to the TF-TRT INT8 inference engine

### Step 1

In [21]:
#Now we create the TFTRT INT8 calibration graph
trt_int8_calib_graph = trt.create_inference_graph(
        input_graph_def=fp32_graph_def,
        outputs=[OUTPUT_TENSOR],
        max_batch_size=BATCH_SIZE,
        max_workspace_size_bytes=1<<32,
        precision_mode='INT8')

#Then calibrate it with 2 batchs of examples
N_runs=2
with tf.Session(graph=tf.Graph()) as sess:
    print('Preparing calibration data...')
    dataset = tf.data.TFRecordDataset(calibration_files)
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    print('Loading INT8 calibration graph...')
    output_node = tf.import_graph_def(
        trt_int8_calib_graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

    print('Calibrate model on calibration data...')
    num_hits = 0
    num_predict = 0
    for _ in range(N_runs):
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run(output_node, feed_dict={INPUT_TENSOR: img})
            prediction = np.argmax(output[0], axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    print('Calibration accuracy: %.2f%%'%(100*num_hits/num_predict)) 


INFO:tensorflow:Running against TensorRT version 5.1.2
Preparing calibration data...
Loading INT8 calibration graph...
Calibrate model on calibration data...
Calibration accuracy: 75.39%


### Step 2

Now we convert the INT8 calibration graph to the final TF-TRT INT8 inference engine, and benchmark its performance. We will also be saving this engine to a *saved model*, ready to be served elsewhere.

In [22]:
#set a directory to write the saved model
INT8_SAVED_MODEL_DIR =  SAVED_MODEL_DIR + "INT8"
!rm -rf $INT8_SAVED_MODEL_DIR


In [23]:
#Create Int8 inference model from the calibration graph and write to a saved session
print(' TF-TRT INT8 inference engine...')
trt_int8_calibrated_graph=trt.calib_graph_to_infer_graph(trt_int8_calib_graph)
output_node = tf.import_graph_def(
        trt_int8_calibrated_graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

with tf.Session(graph=tf.Graph()) as sess:
    print('Preparing dataset...')
    dataset = tf.data.TFRecordDataset(calibration_files)    
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    print('Loading TF-TRT INT8 inference engine...')
    output_node = tf.import_graph_def(
        trt_int8_calibrated_graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

    print('Warming up for 10 batches...')
    for _ in range (10):
        image_data = sess.run(next_element)    
        img = image_data[0]
        output = sess.run([OUTPUT_TENSOR], feed_dict={INPUT_TENSOR: img})
        
    print('Benchmarking TF-TRT INT8 inference engine...')
    num_hits = 0
    num_predict = 0
    start_time = time.time()
    try:
        while True:        
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run([OUTPUT_TENSOR], feed_dict={INPUT_TENSOR: img})
            prediction = np.argmax(output[0], axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    except tf.errors.OutOfRangeError as e:
        pass
            
    print('INT8 TF-TRT Accuracy: %.2f%%'%(100*num_hits/num_predict))
    print('INT8 TF-TRT Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))

    #Save model for serving
    print('Saving INT8 model to %s'%INT8_SAVED_MODEL_DIR)
    tf.saved_model.simple_save(
        session=sess,
        export_dir=INT8_SAVED_MODEL_DIR,
        inputs={"input":tf.get_default_graph().get_tensor_by_name(INPUT_TENSOR)},
        outputs={"softmax":tf.get_default_graph().get_tensor_by_name(OUTPUT_TENSOR)},
        legacy_init_op=None
     )

 TF-TRT INT8 inference engine...
Preparing dataset...
Loading TF-TRT INT8 inference engine...
Warming up for 10 batches...
Benchmarking TF-TRT INT8 inference engine...
INT8 TF-TRT Accuracy: 75.20%
INT8 TF-TRT Inference speed: 1370.55 samples/s
Saving INT8 model to /saved_model/resnet_v1_50_savedmodel/1INT8
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /saved_model/resnet_v1_50_savedmodel/1INT8/saved_model.pb


## Benchmarking INT8 saved model

Finally we reload and verify the performance of the INT8 saved model from disk.

In [24]:
#benchmark int8 saved model
with tf.Session(graph=tf.Graph()) as sess:
    print('Preparing data...')
    dataset = tf.data.TFRecordDataset(calibration_files)    
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=8))
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    print('Loading TF-TRT INT8 inference engine...')
    tf.saved_model.loader.load(
        sess, [tf.saved_model.tag_constants.SERVING], INT8_SAVED_MODEL_DIR)
    
    print('Warming up for 10 batches...')
    for _ in range (10):
        image_data = sess.run(next_element)    
        img = image_data[0]
        output = sess.run([OUTPUT_TENSOR], feed_dict={INPUT_TENSOR: img})
    
    print('Benchmarking TF-TRT INT8 inference engine...')
    num_hits = 0
    num_predict = 0
    start_time = time.time()
    try:
        while True:        
            image_data = sess.run(next_element)    
            img = image_data[0]
            label = image_data[1].squeeze()
            output = sess.run([OUTPUT_TENSOR], feed_dict={INPUT_TENSOR: img})
            prediction = np.argmax(output[0], axis=1)
            num_hits += np.sum(prediction == label)
            num_predict += len(prediction)
    except tf.errors.OutOfRangeError as e:
        pass
            
    print('INT8 TF-TRT Accuracy: %.2f%%'%(100*num_hits/num_predict))
    print('INT8 TF-TRT Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))


Preparing data...
Loading TF-TRT INT8 inference engine...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.
Warming up for 10 batches...
Benchmarking TF-TRT INT8 inference engine...
INT8 TF-TRT Accuracy: 75.20%
INT8 TF-TRT Inference speed: 1352.07 samples/s


In [25]:
!saved_model_cli show --all --dir $INT8_SAVED_MODEL_DIR


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 224, 224, 3)
        name: import/input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['softmax'] tensor_info:
        dtype: DT_FLOAT
        shape: unknown_rank
        name: import/resnet_v1_50/predictions/Softmax:0
  Method name is: tensorflow/serving/predict


## Benchmarking with synthetic data

While benchmarking with real datasets, there are data reading and pre-processing procedures on CPU involved, which might be the bottleneck. As a result, the GPU is not fully loaded all the time. In this section, we test with synthetic data to test the throughput limit of the GPU.

In [26]:
NUM_ITER = 100
dummy_input = np.random.random_sample((BATCH_SIZE,224,224,3))

fp32_graph = tf.Graph()
with fp32_graph.as_default():
    print('Praparing synthetic dataset...')
    inc=tf.constant(dummy_input, dtype=tf.float32)
    dataset=tf.data.Dataset.from_tensors(inc)
    dataset=dataset.repeat()
    iterator=dataset.make_one_shot_iterator()
    next_element=iterator.get_next()
        
    print('Loading model...')
    out = tf.import_graph_def(
      graph_def=fp32_graph_def,
      input_map={INPUT_TENSOR:next_element},
      return_elements=[OUTPUT_TENSOR]
    )

with tf.Session(graph=fp32_graph) as sess:
    print('Warming up for 10 batches...')
    for _ in range(10):
        sess.run(out)
    
    print('Benchmarking...')
    start_time = time.time()
    for _ in range(NUM_ITER):
        sess.run(out)
    print('Naitive FP32 Inference speed: %.2f samples/s'%(NUM_ITER*BATCH_SIZE/(time.time()-start_time)))
    

Praparing synthetic dataset...
Loading model...
Warming up for 10 batches...
Benchmarking...
Naitive FP32 Inference speed: 993.86 samples/s


In [28]:
fp16_graph = tf.Graph()
with fp16_graph.as_default():
    print('Praparing synthetic dataset...')
    inc=tf.constant(dummy_input, dtype=tf.float32)
    dataset=tf.data.Dataset.from_tensors(inc)
    dataset=dataset.repeat()
    iterator=dataset.make_one_shot_iterator()
    next_element=iterator.get_next()
    
    print('Loading model...')
    out = tf.import_graph_def(
      graph_def=trt_fp16_Graph,
      input_map={INPUT_TENSOR:next_element},
      return_elements=[OUTPUT_TENSOR]
    )

with tf.Session(graph=fp16_graph) as sess:
    print('Warming up for 10 batches...')
    for _ in range(10):
        sess.run(out)
    
    print('Benchmarking...')
    start_time = time.time()
    for _ in range(NUM_ITER):
        sess.run(out)
    print('FP16 Inference speed: %.2f samples/s'%(NUM_ITER*BATCH_SIZE/(time.time()-start_time)))
    

Praparing synthetic dataset...
Loading model...
Warming up for 10 batches...
Benchmarking...
FP16 Inference speed: 3357.67 samples/s


In [29]:
int8_graph = tf.Graph()
with int8_graph.as_default():
    print('Praparing synthetic dataset...')
    inc=tf.constant(dummy_input, dtype=tf.float32)
    dataset=tf.data.Dataset.from_tensors(inc)
    dataset=dataset.repeat()
    iterator=dataset.make_one_shot_iterator()
    next_element=iterator.get_next()
    
    print('Loading model...')
    out = tf.import_graph_def(
      graph_def=trt_int8_calibrated_graph,
      input_map={INPUT_TENSOR:next_element},
      return_elements=[OUTPUT_TENSOR],
      name=''        
    )
        
with tf.Session(graph=int8_graph) as sess:

    print('Warming up for 10 batches...')
    for _ in range(10):
        sess.run(out)
    
    print('Benchmarking...')
    start_time = time.time()
    for _ in range(NUM_ITER):
        sess.run(out)
    print('INT8 Inference speed: %.2f samples/s'%(NUM_ITER*BATCH_SIZE/(time.time()-start_time)))

Praparing synthetic dataset...
Loading model...
Warming up for 10 batches...
Benchmarking...
INT8 Inference speed: 3388.26 samples/s
